In [1]:
# todo: playing around with data, we realize we probably want to filter out any trips with negative amounts

# end-to-end, this analysis would look like....

In [2]:
import darkwing as dw

In [3]:
t1 = dw.Table('data/yellow_tripdata_2010-01.parquet')
t1

┌───────────┬─────────────────────┬─────────────────────┬───┬────────────┬──────────────┬──────────────┐
│ vendor_id │   pickup_datetime   │  dropoff_datetime   │ … │ tip_amount │ tolls_amount │ total_amount │
│  varchar  │       varchar       │       varchar       │   │   double   │    double    │    double    │
├───────────┼─────────────────────┼─────────────────────┼───┼────────────┼──────────────┼──────────────┤
│ VTS       │ 2010-01-26 07:41:00 │ 2010-01-26 07:45:00 │ … │        0.0 │          0.0 │          5.0 │
│ DDS       │ 2010-01-30 23:31:00 │ 2010-01-30 23:46:12 │ … │        0.0 │          0.0 │         16.3 │
│ DDS       │ 2010-01-18 20:22:20 │ 2010-01-18 20:38:12 │ … │        0.0 │          0.0 │         12.7 │
│ VTS       │ 2010-01-09 01:18:00 │ 2010-01-09 01:35:00 │ … │        0.0 │          0.0 │         14.3 │
│ CMT       │ 2010-01-18 19:10:14 │ 2010-01-18 19:17:07 │ … │       0.87 │          0.0 │         6.67 │
│ DDS       │ 2010-01-23 18:40:25 │ 2010-01-23 18:54:51

In [4]:
t1.columns

['vendor_id',
 'pickup_datetime',
 'dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'pickup_longitude',
 'pickup_latitude',
 'rate_code',
 'store_and_fwd_flag',
 'dropoff_longitude',
 'dropoff_latitude',
 'payment_type',
 'fare_amount',
 'surcharge',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'total_amount']

In [5]:
# You can use standard duckdb SQL expressions. The `from <table_name>` is automatically inserted for you, so it should be omitted.
# `select ...` is optional. If omitted, `select *` will be inserted.

# convert pickup lat/lngs to H3 cells (at resolution 8)
# resolution 8 cells are about 0.7 km^2 in size
x = t1.do('select *, h3_latlng_to_cell(pickup_latitude, pickup_longitude, 8) as hexid')
x

┌───────────┬─────────────────────┬─────────────────────┬───┬──────────────┬──────────────┬────────────────────┐
│ vendor_id │   pickup_datetime   │  dropoff_datetime   │ … │ tolls_amount │ total_amount │       hexid        │
│  varchar  │       varchar       │       varchar       │   │    double    │    double    │       uint64       │
├───────────┼─────────────────────┼─────────────────────┼───┼──────────────┼──────────────┼────────────────────┤
│ VTS       │ 2010-01-26 07:41:00 │ 2010-01-26 07:45:00 │ … │          0.0 │          5.0 │ 613229522950553599 │
│ DDS       │ 2010-01-30 23:31:00 │ 2010-01-30 23:46:12 │ … │          0.0 │         16.3 │ 613229551343894527 │
│ DDS       │ 2010-01-18 20:22:20 │ 2010-01-18 20:38:12 │ … │          0.0 │         12.7 │ 613229522990399487 │
│ VTS       │ 2010-01-09 01:18:00 │ 2010-01-09 01:35:00 │ … │          0.0 │         14.3 │ 613229524244496383 │
│ CMT       │ 2010-01-18 19:10:14 │ 2010-01-18 19:17:07 │ … │          0.0 │         6.67 │ 6132

In [6]:
# compute the average of `total_amount` for each H3 cell, along with the number of trips
x = x.do('select hexid, avg(total_amount) as amount, count(*) as num group by 1')
x

┌────────────────────┬────────────────────┬────────┐
│       hexid        │       amount       │  num   │
│       uint64       │       double       │ int64  │
├────────────────────┼────────────────────┼────────┤
│ 613229522992496639 │   9.67052480701034 │ 288487 │
│ 613229524244496383 │ 10.816309067189671 │ 666127 │
│ 613229524766687231 │  28.70774723824092 │  71965 │
│ 613229524311605247 │   12.8694763361526 │  11114 │
│ 613229524210941951 │ 10.594611812368376 │ 185619 │
│ 613229522940067839 │ 11.282130774389216 │ 105877 │
│ 613229522954747903 │  8.755456601197542 │ 140626 │
│ 613229551411003391 │ 14.289010873562011 │  38074 │
│ 613229524162707455 │ 12.121935231539425 │   6392 │
│ 613229551402614783 │ 14.603827150293311 │  12103 │
│          ·         │                 ·  │      · │
│          ·         │                 ·  │      · │
│          ·         │                 ·  │      · │
│ 613255592867266559 │               21.5 │      1 │
│ 613229544909832191 │               34.2 │   

In [7]:
# Convert the H3 cell id to its hexadecimal representation
x = x.do('select h3_h3_to_string(hexid) as hexid, amount, num')
x

┌─────────────────┬────────────────────┬────────┐
│      hexid      │       amount       │  num   │
│     varchar     │       double       │ int64  │
├─────────────────┼────────────────────┼────────┤
│ 882a100d65fffff │ 10.816309067189671 │ 666127 │
│ 882a100897fffff │  8.755456601197542 │ 140626 │
│ 882a1008bbfffff │  9.670524807010343 │ 288487 │
│ 882a107299fffff │ 14.289010873562011 │  38074 │
│ 882a100d45fffff │ 10.594611812368376 │ 185619 │
│ 882a100f57fffff │ 28.707747238240927 │  71965 │
│ 882a100889fffff │ 11.282130774389216 │ 105877 │
│ 882a100ebbfffff │  39.36631964809386 │    682 │
│ 882a1008c3fffff │ 12.251557967709575 │  43852 │
│ 882a107291fffff │ 14.603827150293315 │  12103 │
│        ·        │                 ·  │      · │
│        ·        │                 ·  │      · │
│        ·        │                 ·  │      · │
│ 882a106443fffff │               16.0 │      1 │
│ 882a1011a7fffff │                6.3 │      1 │
│ 882a10c1e3fffff │                8.9 │      1 │


In [8]:
# `Table.do()` also selects a sequence of strings, so you could write all of the previous steps like this

t1.do(
    'select *, h3_latlng_to_cell(pickup_latitude, pickup_longitude, 8) as hexid',
    'select hexid, avg(total_amount) as amount, count(*) as num group by 1',
    'select h3_h3_to_string(hexid) as hexid, amount, num',
)


┌─────────────────┬────────────────────┬────────┐
│      hexid      │       amount       │  num   │
│     varchar     │       double       │ int64  │
├─────────────────┼────────────────────┼────────┤
│ 882a10089bfffff │ 10.686034118679826 │ 252882 │
│ 882a1072c7fffff │  11.64960305036337 │ 203779 │
│ 882a100893fffff │  9.661533032430293 │ 426051 │
│ 882a100883fffff │  9.985855459480879 │ 222512 │
│ 882a1072c3fffff │ 11.387625279142469 │  89560 │
│ 882a1072c9fffff │ 10.324864870461472 │ 386333 │
│ 882a100899fffff │  10.31212280293725 │ 210679 │
│ 882a10725bfffff │ 10.288604039526437 │ 421287 │
│ 882a100881fffff │ 10.705657822863861 │  76890 │
│ 882a107281fffff │  14.54941136643694 │  73849 │
│        ·        │                 ·  │      · │
│        ·        │                 ·  │      · │
│        ·        │                 ·  │      · │
│ 882a1078cbfffff │               13.4 │      1 │
│ 882a162b03fffff │                5.4 │      1 │
│ 882a84c495fffff │                4.6 │      1 │


In [9]:
# This seems like a generally useful transformation, so let's store it as a list of strings
avg_list = [
    'select *, h3_latlng_to_cell(pickup_latitude, pickup_longitude, 8) as hexid',
    'select hexid, avg(total_amount) as amount, count(*) as num group by 1',
    'select h3_h3_to_string(hexid) as hexid, amount, num',
]

t1.do(*avg_list)
t1.do(avg_list)

┌─────────────────┬────────────────────┬────────┐
│      hexid      │       amount       │  num   │
│     varchar     │       double       │ int64  │
├─────────────────┼────────────────────┼────────┤
│ 882a100d29fffff │   9.82921158914791 │ 504610 │
│ 882a100d35fffff │ 10.773760522602288 │ 246612 │
│ 882a100d67fffff │ 10.106109311563587 │ 594594 │
│ 882a1008d5fffff │  11.39579412448646 │  25802 │
│ 882a103b15fffff │ 40.142359940847626 │  29077 │
│ 882a1008b9fffff │  9.775451726322805 │ 224118 │
│ 882a100f33fffff │  11.08422301690507 │   3076 │
│ 882a100da3fffff │  13.47626686656671 │   5336 │
│ 882a100d5bfffff │ 13.457712318286152 │  10456 │
│ 882a1008d7fffff │ 10.326151716810196 │  35764 │
│        ·        │                 ·  │      · │
│        ·        │                 ·  │      · │
│        ·        │                 ·  │      · │
│ 882a139731fffff │                7.1 │      1 │
│ 882a1055c1fffff │               39.0 │      1 │
│ 882b8a665dfffff │               12.6 │      1 │


In [10]:
# Making things a little more formal, we could also store the operation as a function

def avg_func(tbl):
    """
    tbl: should have columns ...
    returns columns ...
    """
    # TODO: can it also take in a df? prolly!
    return tbl.do(avg_list)

In [11]:
# the following are equivalent
avg_func(t1)
t1.do(avg_func)
t1.do([avg_func])

┌─────────────────┬────────────────────┬────────┐
│      hexid      │       amount       │  num   │
│     varchar     │       double       │ int64  │
├─────────────────┼────────────────────┼────────┤
│ 882a100897fffff │  8.755456601197542 │ 140626 │
│ 882a100d65fffff │  10.81630906718967 │ 666127 │
│ 882a1008bbfffff │   9.67052480701034 │ 288487 │
│ 882a100da5fffff │ 12.869476336152601 │  11114 │
│ 882a100d45fffff │  10.59461181236837 │ 185619 │
│ 882a100889fffff │ 11.282130774389215 │ 105877 │
│ 882a100f57fffff │ 28.707747238240923 │  71965 │
│ 882a1008c3fffff │  12.25155796770957 │  43852 │
│ 882a107299fffff │ 14.289010873562011 │  38074 │
│ 882a100da7fffff │ 12.546433862433863 │  15120 │
│        ·        │                 ·  │      · │
│        ·        │                 ·  │      · │
│        ·        │                 ·  │      · │
│ 882a102839fffff │                6.2 │      1 │
│ 882a138b65fffff │                6.0 │      1 │
│ 882a10128bfffff │               10.4 │      1 │


In [12]:
# We can build up complex nested expressions by combining functions, strings, and lists.

# The following are equivalent:

t1.do(avg_func, 'where num > 100')
t1.do(avg_list, 'where num > 100')
t1.do([avg_func, 'where num > 100'])

┌─────────────────┬────────────────────┬────────┐
│      hexid      │       amount       │  num   │
│     varchar     │       double       │ int64  │
├─────────────────┼────────────────────┼────────┤
│ 882a100d25fffff │  9.557408611391345 │ 321156 │
│ 882a107259fffff │ 10.414865664934148 │ 102356 │
│ 882a100d21fffff │  9.739929882891428 │ 513712 │
│ 882a100dadfffff │  12.62425013535463 │   1847 │
│ 882a10088bfffff │ 10.179375811939385 │  64660 │
│ 882a100d05fffff │   9.92728901254454 │  61461 │
│ 882a1072d7fffff │ 14.259547345413528 │  14183 │
│ 882a100d3dfffff │ 10.664021710811987 │  46060 │
│ 882a107293fffff │ 13.798980309423346 │  11376 │
│ 882a1072c5fffff │ 11.187289017341051 │  46710 │
│        ·        │          ·         │     ·  │
│        ·        │          ·         │     ·  │
│        ·        │          ·         │     ·  │
│ 882a100ae5fffff │ 15.114158415841583 │    101 │
│ 882a107055fffff │ 22.933666666666667 │    120 │
│ 882a100dbbfffff │ 13.068114285714286 │    175 │


# Joins

We want to compute average fares for hexes and compare them across January and February. We compute the averages like above, but also want to exlude hexes with only a few trips. So we extend the data pipeline to filter out such hexes, and apply the same operation to the datasets for each month.

In [13]:
t1 = dw.Table('data/yellow_tripdata_2010-01.parquet')
t2 = dw.Table('data/yellow_tripdata_2010-02.parquet')

f = [
    'where total_amount > 0',
    avg_list,
    'where num > 100',
]

t1 = t1.do(f)
t2 = t2.do(f)

t2

┌─────────────────┬────────────────────┬────────┐
│      hexid      │       amount       │  num   │
│     varchar     │       double       │ int64  │
├─────────────────┼────────────────────┼────────┤
│ 882a1008bbfffff │  9.839421424666112 │ 204848 │
│ 882a100f57fffff │  30.09185805984283 │  49697 │
│ 882a100897fffff │  8.916072988533125 │ 105085 │
│ 882a100889fffff │ 11.408574980420441 │  80441 │
│ 882a100ebbfffff │  53.30473029045642 │    482 │
│ 882a100da5fffff │ 13.302083199999995 │   9375 │
│ 882a107299fffff │ 14.523038287960398 │  27476 │
│ 882a100d65fffff │ 10.919830938548953 │ 494199 │
│ 882a100d45fffff │ 10.683853485818956 │ 137543 │
│ 882a100da7fffff │ 12.860021669411465 │  11537 │
│        ·        │          ·         │     ·  │
│        ·        │          ·         │     ·  │
│        ·        │          ·         │     ·  │
│ 882a100d1bfffff │ 12.530097087378639 │    206 │
│ 882a100dd5fffff │ 13.044615384615385 │    117 │
│ 882a100dcbfffff │ 12.323053435114502 │    131 │


To perform a join, you need two tables in on place. In Darkwing, such a place is a Database.

In [14]:
db = dw.Database(t1=t1, t2=t2)

In [15]:
db['t1']

┌─────────────────┬────────────────────┬────────┐
│      hexid      │       amount       │  num   │
│     varchar     │       double       │ int64  │
├─────────────────┼────────────────────┼────────┤
│ 882a100d67fffff │ 10.106109311563587 │ 594594 │
│ 882a100d29fffff │  9.829211589147905 │ 504610 │
│ 882a100d35fffff │ 10.773760522602283 │ 246612 │
│ 882a103b15fffff │  40.14235994084762 │  29077 │
│ 882a100ae7fffff │ 17.057128245476008 │   2542 │
│ 882a1008b9fffff │  9.775451726322805 │ 224118 │
│ 882a100da3fffff │ 13.476266866566714 │   5336 │
│ 882a1072d5fffff │ 13.100060658578862 │   6924 │
│ 882a107745fffff │ 13.864570184983677 │    919 │
│ 882a100d37fffff │  11.11603161144446 │  45996 │
│        ·        │          ·         │     ·  │
│        ·        │          ·         │     ·  │
│        ·        │          ·         │     ·  │
│ 882a103b19fffff │ 29.437169811320757 │    106 │
│ 882a1008e9fffff │ 17.742466666666665 │    150 │
│ 882a107287fffff │  10.71086092715232 │    151 │


In [16]:
# but note that because evaluation is lazy, it will recompute each time.
# If you want to avoid that, you can materialize the computation and create a new database. Note there is no need to do this if you don't mind recomputing.

In [17]:
db = db.hold()

In [18]:
# NOTE: maybe confusing to users who now see a new view of their table...
# I wonder if caching the relation, or making it a property is the way to go...
db['t1']

pyarrow.Table
hexid: string
amount: double
num: int64
----
hexid: [["882a100893fffff","882a10725bfffff","882a1072c9fffff","882a1072c7fffff","882a1072c3fffff",...,"882a100c35fffff","882a10745dfffff","882a107621fffff","882a107737fffff","882a1008adfffff"]]
amount: [[9.661533032430288,10.288604039526438,10.324864870461473,11.649603050363378,11.38762527914247,...,11.5295,11.348403361344536,11.974796747967481,10.145294117647058,11.878911564625852]]
num: [[426051,421287,386333,203779,89560,...,180,119,123,102,147]]

In [19]:
# You can execute duckdb sql on a database, but now you should explicitly mention the table you want to work with (but that's usually what you want anyway, when doing a join.)

In [22]:
db.do("""
select
      hexid
    , t1.amount
        as amount1
    , t2.amount
        as amount2
from
    t1
inner join
    t2
using
    (hexid)
""").do("""
select
      *
    , amount2 - amount1
        as change
order by
    change
""")

┌─────────────────┬────────────────────┬────────────────────┬─────────────────────┐
│      hexid      │      amount1       │      amount2       │       change        │
│     varchar     │       double       │       double       │       double        │
├─────────────────┼────────────────────┼────────────────────┼─────────────────────┤
│ 882a1071adfffff │  69.73084615384616 │ 58.737914798206276 │ -10.992931355639882 │
│ 882a100c01fffff │ 27.110503597122303 │  20.74137254901961 │  -6.369131048102695 │
│ 882a100e37fffff │  33.03063551401869 │ 28.065557655954635 │  -4.965077858064056 │
│ 882a1072e7fffff │  42.25520958083832 │  38.42423076923076 │  -3.830978811607558 │
│ 882a1008e9fffff │ 17.742466666666665 │ 14.009843750000002 │ -3.7326229166666636 │
│ 882a1008e7fffff │ 14.852932330827066 │ 11.121284403669724 │ -3.7316479271573417 │
│ 882a107207fffff │ 16.860051282051284 │ 13.165116279069768 │ -3.6949350029815164 │
│ 882a100e85fffff │ 29.665849999999992 │  26.64545454545455 │   -3.020395454